In [51]:
import cv2
import numpy as np
cam_http_addr = "http://149.156.199.121"
vid = cv2.VideoCapture("{}/axis-cgi/mjpg/video.cgi?resolution=640x480&compression=70&fps=30".format(cam_http_addr))
(status, frame) = vid.read()
average = np.float32(frame) 
alpha = 0.01
while True:
    (status, frame) = vid.read()
    cv2.accumulateWeighted(frame, average, alpha) 
    result = cv2.convertScaleAbs(average)
    diff = cv2.absdiff(frame,result)
    cv2.imshow('Stream', frame)
    cv2.imshow('Background', result) 
    cv2.imshow('Diff', diff)
    if cv2.waitKey(1) == ord('q'):
        break
vid.release()
cv2.destroyAllWindows()

In [40]:
import cv2
import numpy as np
cam_http_addr = "http://149.156.199.121"
vid = cv2.VideoCapture("{}/axis-cgi/mjpg/video.cgi?resolution=640x480&compression=70&fps=30".format(cam_http_addr))

backSub = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=16, detectShadows=False)

while True:
    ret, frame = vid.read()
        
    fgMask = backSub.apply(frame)
    background = backSub.getBackgroundImage()
    cv2.imshow('Frame', frame)
    cv2.imshow('Foreground Mask', fgMask)
    cv2.imshow('Background', background)
    
    if cv2.waitKey(1) == ord('q'):
        break

vid.release()
cv2.destroyAllWindows()


In [44]:
import cv2
import numpy as np
cam_http_addr = "http://149.156.199.121"
vid = cv2.VideoCapture("{}/axis-cgi/mjpg/video.cgi?resolution=640x480&compression=70&fps=30".format(cam_http_addr))

backSub = cv2.createBackgroundSubtractorKNN(history=500,dist2Threshold=200,detectShadows=False)

while True:
    ret, frame = vid.read()
        
    fgMask = backSub.apply(frame)
    background = backSub.getBackgroundImage()
    cv2.imshow('Frame', frame)
    cv2.imshow('Foreground Mask', fgMask)
    cv2.imshow('Background', background)
    
    if cv2.waitKey(1) == ord('q'):
        break

vid.release()
cv2.destroyAllWindows()


In [3]:
import cv2
import numpy as np
import os

cam_http_addr = "http://149.156.199.121"
vid = cv2.VideoCapture("{}/axis-cgi/mjpg/video.cgi?resolution=640x480&compression=70&fps=30".format(cam_http_addr))

backSub_MOG2 = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=50, detectShadows=False)
backSub_KNN = cv2.createBackgroundSubtractorKNN(history=500, dist2Threshold=300, detectShadows=False)

(status, frame) = vid.read()
average = np.float32(frame) 
alpha = 0.01

output_dir = "background_results_street"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out_comparison = cv2.VideoWriter(os.path.join(output_dir, "comparison.avi"), fourcc, 30.0, (frame.shape[1] * 3, frame.shape[0]))
out_comparison_diff = cv2.VideoWriter(os.path.join(output_dir, "diff.avi"), fourcc, 30.0, (frame.shape[1] * 3, frame.shape[0]))

while True:
    ret, frame = vid.read()
    
    if not ret:
        break
    
    cv2.accumulateWeighted(frame, average, alpha) 
    average_result = cv2.convertScaleAbs(average)

    fgMask_MOG2 = backSub_MOG2.apply(frame)
    background_MOG2 = backSub_MOG2.getBackgroundImage()
    
    fgMask_KNN = backSub_KNN.apply(frame)
    background_KNN = backSub_KNN.getBackgroundImage()

    comparison = np.hstack((average_result, background_MOG2, background_KNN))
    cv2.imshow('Comparison (Avg / MOG2 / KNN)', comparison)

    out_comparison.write(comparison)

    diff_avg = cv2.absdiff(frame, average_result)
    diff_MOG2 = cv2.absdiff(frame, background_MOG2) if background_MOG2 is not None else np.zeros_like(frame)
    diff_KNN = cv2.absdiff(frame, background_KNN) if background_KNN is not None else np.zeros_like(frame)

    diff_comparison = np.hstack((diff_avg, diff_MOG2, diff_KNN))
    cv2.imshow('Difference (Avg / MOG2 / KNN)', diff_comparison)

    out_comparison_diff.write(diff_comparison)

    if cv2.waitKey(1) == ord('q'):
        break

vid.release()
cv2.destroyAllWindows()
